# Plan A 

In [2]:
sample = 'dabAcCaCBAcCcaDA'

In [3]:
polymer = sample[:]
output = ''
i = 0
while i < len(polymer)-1:
    u, v = polymer[i], polymer[i + 1]

    if u.lower() == v.lower() and u != v:
        # reaction
        left = polymer[:i]
        right = polymer[i+2:]
        polymer = left + right
        # move on
        i = len(left) - 1
        
    else:
        # no reaction
        i += 1
    #print(i, u, v, polymer)
len(polymer)

10

In [4]:
polymer == 'dabCBAcaDA'

True

Let's now try with the real input.

In [5]:
real_polymer = [line.strip() for line in open('input05').readlines()][0]

In [6]:
real_polymer[:10]
real_polymer[-10:]

'tTRroUkKuI'

In [7]:
def react_until_end_fast(polymer):
    i = 0
    while i < len(polymer)-1:
        u, v = polymer[i], polymer[i + 1]

        if u.lower() == v.lower() and u != v:
            # reaction
            left = polymer[:i]
            right = polymer[i+2:]
            assert(len(left + right) < len(polymer))
            polymer = left + right

            # move on
            i = len(left) - 1
            if i < 0:
                i = 0
        else:
            # no reaction
            i += 1
        #print(i, u, v, polymer)
    len(polymer)
    return polymer

In [8]:
len(react_until_end_fast(real_polymer))

9686

# Plan B 

It seems that I'm stuck here. Let's try plan B.

We will write a function that scans the chain from its start and creates the next chain by reacting the first element it finds.

In [8]:
def react_once(polymer):
    i = 0
    while i < len(polymer)-1:
        u, v = polymer[i], polymer[i + 1]

        if u.lower() == v.lower() and u != v:
            # reaction
            left = polymer[:i]
            right = polymer[i+2:]
            assert(len(left + right) < len(polymer))
            polymer = left + right
            return polymer
        else:
            # no reaction
            i += 1
    return polymer

In [9]:
react_once('aA')

''

In [10]:
react_once('abBA')

'aA'

In [11]:
react_once('abAB')

'abAB'

In [12]:
react_once('aabAAB')

'aabAAB'

In [13]:
react_once('dabAcCaCBAcCcaDA')

'dabAaCBAcCcaDA'

Now let's write a function that reacts until the reaction can not take place anymore.

In [14]:
def react_until_end(polymer):
    """Super slow!"""
    n = len(polymer)
    polymer = react_once(polymer)
    while len(polymer) < n:
        n = len(polymer)
        polymer = react_once(polymer)
    return polymer

In [15]:
react_until_end('dabAcCaCBAcCcaDA')

'dabCBAcaDA'

In [16]:
len('dabCBAcaDA')

10

In [17]:
len(react_until_end(real_polymer))

9686

# Part B 

Let's write a function that replaces a unit.

In [9]:
def remove_unit(polymer, unit_type):
    return polymer.replace(unit_type.lower(), '').replace(unit_type.upper(), '')

In [10]:
remove_unit('dabAcCaCBAcCcaDA', 'a')

'dbcCCBcCcD'

In [11]:
for unit_type in 'abcd':
    polymer = remove_unit('dabAcCaCBAcCcaDA', unit_type)
    reacted = react_until_end_fast(polymer)
    print(unit_type, len(reacted))

a 6
b 8
c 4
d 6


In [15]:
lens = []
for unit_type in list(set(real_polymer.lower())):
    polymer = remove_unit(real_polymer, unit_type)
    reacted = react_until_end_fast(polymer)
    lens.append(len(reacted))
    print(unit_type, len(reacted))
min(lens)

j 9336
s 9284
p 9322
o 9320
q 9306
b 9368
l 9314
x 9368
v 9260
i 9276
g 9314
k 5524
z 9268
t 9250
e 9304
m 9228
y 9278
n 9274
u 9280
c 9262
h 9316
r 9326
f 9316
w 9364
a 9272
d 9282


5524

# Benchmark 

In [16]:
%%time
lens = []
for unit_type in list(set(real_polymer.lower())):
    polymer = remove_unit(real_polymer, unit_type)
    reacted = react_until_end_fast(polymer)
    lens.append(len(reacted))
    print(unit_type, len(reacted))
min(lens)

j 9336
s 9284
p 9322
o 9320
q 9306
b 9368
l 9314
x 9368
v 9260
i 9276
g 9314
k 5524
z 9268
t 9250
e 9304
m 9228
y 9278
n 9274
u 9280
c 9262
h 9316
r 9326
f 9316
w 9364
a 9272
d 9282
CPU times: user 3.41 s, sys: 40.2 ms, total: 3.45 s
Wall time: 3.47 s


In [17]:
from numba import njit

In [19]:
react_numba = njit(react_until_end_fast)

In [20]:
react_numba(polymer)

TypingError: Failed at nopython (nopython frontend)
Invalid usage of Function(<built-in function len>) with parameters (str)
 * parameterized
File "<ipython-input-7-8bc6d805dda3>", line 3
[1] During: resolving callee type: Function(<built-in function len>)
[2] During: typing of call at <ipython-input-7-8bc6d805dda3> (3)

In [24]:
from pythran.magic import load_ipython_extension

In [25]:
load_ipython_extension()e

TypeError: load_ipython_extension() missing 1 required positional argument: 'ipython'

In [22]:
help(magic)

Help on module pythran.magic in pythran:

NAME
    pythran.magic - Pythran integration into IPython.

DESCRIPTION
    * provides the %%pythran magic function to ipython

CLASSES
    IPython.core.magic.Magics(traitlets.config.configurable.Configurable)
        PythranMagics
    
    class PythranMagics(IPython.core.magic.Magics)
     |  Class to make it possible to use pythran as a magic IPython command.
     |  
     |  Method resolution order:
     |      PythranMagics
     |      IPython.core.magic.Magics
     |      traitlets.config.configurable.Configurable
     |      traitlets.traitlets.HasTraits
     |      traitlets.traitlets.HasDescriptors
     |      builtins.object
     |  
     |  Methods defined here:
     |  
     |  __init__(self, shell)
     |      Init the pythran magic stuff.
     |  
     |  pythran(self, line, cell)
     |      ::
     |      
     |        %pythran [-D D] [-O O] [-march MARCH] [-fopenmp]
     |      
     |      Compile and import everything from a

In [ ]:
magic

In [28]:
import pythran

In [30]:
%%pythran.magic

UsageError: Cell magic `%%pythran.magic` not found.
